In [1]:
import tensorflow as tf
import sys
from tensorflow.python.platform import gfile
def read_data(src_path):
    data_set = []
    counter = 0
    max_length1 = 0
    with tf.io.gfile.GFile(src_path, mode="r") as src_file:
        src = src_file.readline()
        while src:
            if counter % 100000 == 0:
                print("  reading data line %d" % counter)
                sys.stdout.flush()

            sentences = []
            s = []
            for x in src.split(" "):
                id = int(x)
                if id != -1:
                    s.append(id)
                else:
                    if len(s) > max_length1:
                        max_length1 = len(s)
                    sentences.append(s)
                    s = []

            data_set.append(sentences)
            counter += 1
            src = src_file.readline()
    print(counter)
    print(max_length1)
    return data_set
    
    
train_data = read_data("data/train.ids")
valid_data = read_data("data/valid.ids")

  reading data line 0
78529
21
  reading data line 0
9816
21


In [2]:
def initialize_vocabulary(vocabulary_path):
  if gfile.Exists(vocabulary_path):
    rev_vocab = []
    with gfile.GFile(vocabulary_path, mode="r") as f:
      rev_vocab.extend(f.readlines())
    rev_vocab = [line.strip() for line in rev_vocab]
    vocab = dict([(x, y) for (y, x) in enumerate(rev_vocab)])
    return vocab, rev_vocab
  else:
    raise ValueError("Vocabulary file %s not found.", vocabulary_path)

In [3]:
def get_pred(path):
    prediction=[]
    file = open(path,'r') #'./output/predict2_file105000'
    line = file.read().splitlines()
    while line:
        prediction.append(line)
        line = file.read().splitlines()
    file.close()
    return prediction  #Using prediction[0][i] to see each sentence

In [4]:
empty_string = "___________________________________ ."
def convert_pred_to_story(data,pred_sentence,converter):
    word = []
    output = []
    pred_story = []
    for i in range (0,len(pred_sentence[0])):
        for j in range (0,len(data[i])):
            for k in range (0,len(data[i][j])):
                word.append(tf.compat.as_str(converter[data[i][j][k]]))
            new_sentence = " ".join(word)
            if j == i % 5:
                
                output.append(empty_string)
            else:
                output.append(new_sentence)

            word.clear()
        new_stroy = " ".join(output)
        pred_story.append(new_stroy)
        output.clear()


    return pred_story

In [5]:
valid_data = read_data("data/valid.ids")
to_vocab, rev_to_vocab = initialize_vocabulary("data/vocab_20000")
pred_sentence_cvae = get_pred('pred_ref_data/predict2_file288000')
pred_sentence_cvae_gan = get_pred('pred_ref_data/predict2_file372000')
ref_sentence_cvae = get_pred('pred_ref_data/ref2_file288000')
ref_sentence_cvae_gan = get_pred('pred_ref_data/ref2_file372000')
ref_story_cvae = convert_pred_to_story(valid_data,ref_sentence_cvae,rev_to_vocab)
ref_story_cvae_gan = convert_pred_to_story(valid_data,ref_sentence_cvae_gan,rev_to_vocab)

  reading data line 0
9816
21


In [6]:
class color:
   BLUE = '\033[94m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   END = '\033[0m'
#For T-CVAE  BLEU = 4.16
for i in range (0,8):
    print(color.BOLD + "story" + color.END +": %s \n" %(ref_story_cvae[i]))
    print(color.BOLD + "answer" + color.END + ": %s \n" %(ref_sentence_cvae[0][i]))
    print(color.BOLD + "predict" + color.END + ": %s \n" %(pred_sentence_cvae[0][i]))
    print("\n")

story: ___________________________________ when i got to the stop sign , the check engine light started flashing . i panicked and carefully drove the van to the nearest mechanic shop . they checked it out but could not repair the van . the van had to be sold for parts and i had to get a new vehicle . 

answer: one day , i was driving the van down to the store . 

predict: i was driving my van down the street one day . 



story: krista was organizing her office . ___________________________________ they were big and heavy . she assembled them carefully . when she put all her books on them , they collapsed ! 

answer: she bought some new shelves . 

predict: she had a bunch of books . 



story: the man won a contest . he went to the station to collect . ___________________________________ he did n't really like the band . he tried to sell them back to the radio employees . 

answer: he was given a set of tickets . 

predict: he got a ticket for a band . 



story: billy is bored . bill

In [7]:
#For T-CVAE-GAN BLEU = 4.46
for i in range (0,8):
    print(color.BOLD + "story" + color.END +": %s \n" %(ref_story_cvae_gan[i]))
    print(color.BOLD + "answer" + color.END + ": %s \n" %(ref_sentence_cvae_gan[0][i]))
    print(color.BOLD + "predict" + color.END + ": %s \n" %(pred_sentence_cvae_gan[0][i]))
    print("\n")

story: ___________________________________ when i got to the stop sign , the check engine light started flashing . i panicked and carefully drove the van to the nearest mechanic shop . they checked it out but could not repair the van . the van had to be sold for parts and i had to get a new vehicle . 

answer: one day , i was driving the van down to the store . 

predict: i was driving to work one day when i got a flat tire . 



story: krista was organizing her office . ___________________________________ they were big and heavy . she assembled them carefully . when she put all her books on them , they collapsed ! 

answer: she bought some new shelves . 

predict: she was making cookies . 



story: the man won a contest . he went to the station to collect . ___________________________________ he did n't really like the band . he tried to sell them back to the radio employees . 

answer: he was given a set of tickets . 

predict: he bought a bunch of tickets . 



story: billy is bore

In [8]:
for i in range (0,8):
    print(color.BOLD + "story" + color.END +": %s \n" %(ref_story_cvae_gan[i]))
    print(color.BOLD + "answer" + color.END + ": %s \n" %(ref_sentence_cvae_gan[0][i]))
    print(color.BOLD + "T-CVAE predict" + color.END + ": %s \n" %(pred_sentence_cvae[0][i]))
    print(color.BOLD + "T-CVAE-GAN predict" + color.END + ": %s \n" %(pred_sentence_cvae_gan[0][i]))
    print("\n")

story: ___________________________________ when i got to the stop sign , the check engine light started flashing . i panicked and carefully drove the van to the nearest mechanic shop . they checked it out but could not repair the van . the van had to be sold for parts and i had to get a new vehicle . 

answer: one day , i was driving the van down to the store . 

T-CVAE predict: i was driving my van down the street one day . 

T-CVAE-GAN predict: i was driving to work one day when i got a flat tire . 



story: krista was organizing her office . ___________________________________ they were big and heavy . she assembled them carefully . when she put all her books on them , they collapsed ! 

answer: she bought some new shelves . 

T-CVAE predict: she had a bunch of books . 

T-CVAE-GAN predict: she was making cookies . 



story: the man won a contest . he went to the station to collect . ___________________________________ he did n't really like the band . he tried to sell them back t